## Клиника СМТ

In [1]:
from urllib.request import urlopen, urlretrieve
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import time
import requests 
from fake_headers import Headers




In [2]:
directions = {} # возьмем все ссылки для медицинский направлений
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'} # для запроса через requests

resp = urlopen('https://clinic-complex.ru/price/')
html = resp.read().decode('utf8') 
soup = BeautifulSoup(html, 'html.parser')

for li in soup.find_all('li'):
    if li.has_attr('class') and ''.join(li.get('class')) == 'itemchild':
        directions.setdefault(li.find_next('a').text, ['https://clinic-complex.ru' + li.find_next('a').get('href'), {}])

for key, val in directions.items():
    pattern = r'{}'.format(re.escape(val[0])) + r'.+/'
    pattern_price = r'\d{1+}'
    resp = urlopen(val[0])
    html = resp.read().decode('utf8') 
    soup = BeautifulSoup(html, 'html.parser')
    for link in soup.find_all('a'):
        if re.search(pattern, 'https://clinic-complex.ru' + link.get('href')) and 'apply' not in link.get('href'):
            time.sleep(1)
            try: # если не получится через urloprn пробуем requests
                resp = urlopen('https://clinic-complex.ru' + link.get('href'))
                html = resp.read().decode('utf8')
                soup = BeautifulSoup(html, 'html.parser')
            except HTTPError as err:
                header = Headers(headers=True)
                headers = header.generate()  
                print('https://clinic-complex.ru' + link.get('href'), err.code)
                html = requests.get('https://clinic-complex.ru' + link.get('href'), headers=headers)
                soup = BeautifulSoup(html, 'html.parser')
                
            if soup.find("div", class_="detail_wrap"):
                b_div = soup.find("div", class_="detail_wrap").find('b').text
                val[1].setdefault(link.text, ''.join([i for i in b_div if i.isdigit()]))

smt_data  = {
            'description' : [],
            'price' : [],
            'department' : []}

for key, val in directions.items():
    for key1, val1 in val[1].items():
        smt_data['description'].append(key1)
        smt_data['price'].append(val1)
        smt_data['department'].append(key)
        
     
data_smt = pd.DataFrame.from_dict(smt_data)
data_smt.columns = ['Описание услуги', 'Цена в рублях', 'Направление']
data_smt['Цена в рублях'] = data_smt['Цена в рублях'].apply(lambda x: float(x.replace(' ', '')))
data_smt.to_excel(r'C:\Users\AmanovRA\Anaconda3\Work-work\Parsings\СМТ_клиника\SMT_clinic_' + str(datetime.date(datetime.today())) + '.xlsx')        
                    

